In [5]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GroupShuffleSplit


from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts
import run_expts

# Using Load results 1 as a base - here I load all of the results to create necessary tables and do statistical significance tests
import calculate_results #This module will help me to calculate the per fold metrics

# Set display options to show all rows and columns
pd.set_option('display.max_rows', 50)  # Show rows
pd.set_option('display.max_columns', 160)  # Show columns

/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/antropy/fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((types.Array(types.float64, 1, "C", readonly=True), types.int32))
/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package yasa is out of date. Your version is 0.6.3, the latest is 0.6.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
#I am adjusting the notebook to save the per fold distributions for the best
# 15 models, 5 data types x (Stat. features, Connectivity (PLI), Stat. features & PLI)

#### Load connectivity data and generate table for all connectivity data

In [24]:
%%capture

methods = ['coh', 'imcoh', 'plv', 'ciplv', 'ppc', 'pli', 'dpli', 'wpli', 'wpli2_debiased'] #removed cohy because it's not there in results folder

data_types = ['REM', 'N1', 'N2', 'N3', 'Wake']
expt_num = 1
save_folder = '/user/home/ko20929/work/RBD_using_custom_package/Execute New Experiments/Connectivity/Connectivity_Results/Experiment_set_2/'

#Try loading a results file
method = methods[0]
data_type = data_types[1]

# Create summary results dictionary that will be filled and populated
summary_results_dict = {}
summary_results_dict['metric'] = []
for data_type in data_types:
    summary_results_dict[data_type + '_model'] = []
    summary_results_dict[data_type + '_acc'] = []
    summary_results_dict[data_type + '_acc' + '_std'] = []

    summary_results_dict[data_type + '_prec'] = []
    summary_results_dict[data_type + '_prec' + '_std'] = []

    summary_results_dict[data_type + '_rec'] = []
    summary_results_dict[data_type + '_rec' + '_std'] = []

#Dictionary to store raw results for calculation of distribution(s) and statistical tests 
raw_results = {}
for method in methods:
    raw_results[method] = {}


#error dict can catch those metrics / data types for which we don't have results files
error_dict = {}
# for method in methods:
# editing this to generate the results dictionary for pli for statistical testing
for method in ['pli']:
    error_dict[method] = []
    
    summary_results_dict['metric'].append(method)

    pli_results_dict = {}
    
    pli_test = [] #to test that there were not multiple cols selected for best classifier
    
    for data_type in data_types:
        try:
            pli_results_dict[data_type] = {}
            
            load_path = save_folder + data_type + method + '_expt_' + str(expt_num) + '_' + 'raw_clfs_results.h5'
            
            results_df = pd.read_hdf(load_path)
            results_df = results_df[ ['random_state', 'y_true'] + [col for col in results_df.columns if 'Ada' in col] ]
            
            raw_results[method][data_type] = results_df

        
            
            res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)
            # This will get you best performing Ada Model and Value for each method for each data type
            best_acc = res_df.mean(axis = 0)[1:].max()
            best_model = res_df.mean(axis = 0)[1:].idxmax()

            #Use the summary res_df to identify best model, keep raw results of best model to calculate the true std
            results_df_trimmed = results_df[ list(results_df.columns)[:2] + [col for col in results_df.columns if best_model[:-4] in col] ]
            per_fold_metrics = calculate_results.calc_per_fold_metrics(results_df_trimmed)
            overalled_weighted_df = calculate_results.overall_weighted_metrics(per_fold_metrics)

            # Populating the results dict for sig. testing
            pli_results_dict[data_type]['acc'] = per_fold_metrics[[col for col in per_fold_metrics.columns if 'acc' in col]].values.ravel()
            pli_results_dict[data_type]['prec'] = per_fold_metrics[[col for col in per_fold_metrics.columns if 'acc' in col]].values.ravel()
            pli_results_dict[data_type]['recall'] = per_fold_metrics[[col for col in per_fold_metrics.columns if 'recall' in col]].values.ravel()
            
            pli_test.append( (data_type, len(per_fold_metrics.columns) ))
            
            acc_std = overalled_weighted_df[best_model + '_std'].values[0]

            prec = overalled_weighted_df[best_model[:-4]+'_prec_mean'].values[0]
            prec_std = overalled_weighted_df[best_model[:-4]+'_prec_std'].values[0]
            rec = overalled_weighted_df[best_model[:-4]+'_recall_mean'].values[0]
            rec_std = overalled_weighted_df[best_model[:-4]+'_recall_std'].values[0]

            summary_results_dict[data_type + '_model'].append(best_model[:-10])
            summary_results_dict[data_type + '_acc'].append(best_acc)
            summary_results_dict[data_type + '_acc' + '_std'].append(acc_std)

            summary_results_dict[data_type + '_prec'].append(prec)
            summary_results_dict[data_type + '_prec' + '_std'].append(prec_std)

            summary_results_dict[data_type + '_rec'].append(rec)
            summary_results_dict[data_type + '_rec' + '_std'].append(rec_std)
            
        except:
            error_dict[method].append(data_type)

In [10]:
# For each method I want to create a dictionary for statistical testing that says: 
# clf_dict e.g. PLI_dict = {'Wake' : {acc : [10 vals] , prec: [10 vals] , recall : [10 vals] } , 
# 'N1' : {acc : [10 vals] , prec: [10 vals] , recall : [10 vals] },
# 'N2' :  {acc : [10 vals] , prec: [10 vals] , recall : [10 vals] } , 'N3' : {acc : [10 vals] , prec: [10 vals] , recall : [10 vals] } , 
# 'REM' : {acc : [10 vals] , prec: [10 vals] , recall : [10 vals] }

In [26]:
pli_test

[('REM', 5), ('N1', 5), ('N2', 5), ('N3', 5), ('Wake', 5)]

In [29]:
joblib.dump(pli_results_dict, 'pli_best_results_dict.pkl')

['pli_best_results_dict.pkl']

In [27]:
pli_results_dict

{'REM': {'acc': array([0.5  , 0.75 , 0.75 , 0.75 , 0.375, 0.75 , 0.625, 0.5  , 0.625,
         0.75 ]),
  'prec': array([0.5  , 0.75 , 0.75 , 0.75 , 0.375, 0.75 , 0.625, 0.5  , 0.625,
         0.75 ]),
  'recall': array([0.5       , 0.66666667, 0.33333333, 1.        , 0.        ,
         1.        , 0.6       , 0.        , 0.33333333, 0.33333333])},
 'N1': {'acc': array([0.625, 0.875, 0.875, 1.   , 0.875, 0.875, 1.   , 0.875, 0.75 ,
         0.875]),
  'prec': array([0.625, 0.875, 0.875, 1.   , 0.875, 0.875, 1.   , 0.875, 0.75 ,
         0.875]),
  'recall': array([1.        , 0.8       , 0.75      , 1.        , 0.        ,
         1.        , 1.        , 0.66666667, 0.5       , 0.66666667])},
 'N2': {'acc': array([0.77777778, 0.75      , 0.75      , 0.5       , 0.75      ,
         0.66666667, 0.5       , 0.75      , 0.75      , 0.5       ]),
  'prec': array([0.77777778, 0.75      , 0.75      , 0.5       , 0.75      ,
         0.66666667, 0.5       , 0.75      , 0.75      , 0.5     

In [7]:
summary_results_df = pd.DataFrame(summary_results_dict)
summary_results_df = summary_results_df.round(3)
# Function to apply bold style
def bold_max(s):
    is_max = s == s.max()
    return ['font-weight: bold' if v else '' for v in is_max]

(summary_results_df.style.apply(bold_max, subset=pd.IndexSlice[:, [col for col in summary_results_df.columns if col.endswith('_acc')]]) )

,metric,REM_model,REM_acc,REM_acc_std,REM_prec,REM_prec_std,REM_rec,REM_rec_std,N1_model,N1_acc,N1_acc_std,N1_prec,N1_prec_std,N1_rec,N1_rec_std,N2_model,N2_acc,N2_acc_std,N2_prec,N2_prec_std,N2_rec,N2_rec_std,N3_model,N3_acc,N3_acc_std,N3_prec,N3_prec_std,N3_rec,N3_rec_std,Wake_model,Wake_acc,Wake_acc_std,Wake_prec,Wake_prec_std,Wake_rec,Wake_rec_std
0,coh,theta,0.600000,0.167000,0.423000,0.349000,0.393000,0.233000,all,0.638000,0.143000,0.517000,0.270000,0.500000,0.259000,all,0.646000,0.130000,0.520000,0.300000,0.433000,0.279000,delta,0.621000,0.090000,0.483000,0.278000,0.467000,0.322000,gamma,0.683000,0.159000,0.609000,0.287000,0.583000,0.282000
1,imcoh,theta,0.575000,0.151000,0.350000,0.293000,0.250000,0.299000,sigma,0.612000,0.104000,0.481000,0.282000,0.433000,0.240000,theta,0.635000,0.173000,0.500000,0.271000,0.667000,0.276000,delta,0.738000,0.168000,0.618000,0.364000,0.700000,0.308000,delta,0.650000,0.140000,0.552000,0.205000,0.667000,0.417000
2,plv,gamma,0.575000,0.140000,0.364000,0.256000,0.286000,0.338000,beta,0.600000,0.123000,0.462000,0.331000,0.400000,0.246000,all,0.625000,0.193000,0.481000,0.362000,0.433000,0.312000,theta,0.619000,0.120000,0.478000,0.306000,0.367000,0.391000,all,0.600000,0.171000,0.500000,0.394000,0.375000,0.397000
3,ciplv,gamma,0.650000,0.110000,0.500000,0.330000,0.393000,0.253000,beta,0.812000,0.116000,0.759000,0.219000,0.733000,0.251000,alpha,0.667000,0.256000,0.533000,0.409000,0.533000,0.290000,all_features,0.668000,0.130000,0.556000,0.253000,0.500000,0.254000,all_features,0.650000,0.140000,0.588000,0.378000,0.417000,0.295000
4,ppc,gamma,0.638000,0.154000,0.480000,0.310000,0.429000,0.337000,beta,0.638000,0.182000,0.522000,0.430000,0.400000,0.368000,beta,0.636000,0.101000,0.500000,0.302000,0.500000,0.260000,theta,0.642000,0.138000,0.522000,0.277000,0.400000,0.308000,gamma,0.650000,0.118000,0.571000,0.217000,0.500000,0.381000
5,pli,theta,0.638000,0.131000,0.480000,0.282000,0.429000,0.304000,gamma,0.862000,0.104000,0.852000,0.256000,0.767000,0.226000,alpha,0.669000,0.114000,0.556000,0.257000,0.500000,0.323000,theta,0.647000,0.165000,0.519000,0.289000,0.467000,0.298000,all_features,0.567000,0.171000,0.458000,0.266000,0.458000,0.304000
6,dpli,theta,0.600000,0.167000,0.389000,0.274000,0.250000,0.374000,all,0.600000,0.185000,0.458000,0.313000,0.367000,0.306000,alpha,0.719000,0.190000,0.630000,0.253000,0.567000,0.342000,theta,0.636000,0.140000,0.500000,0.320000,0.533000,0.313000,delta,0.550000,0.226000,0.435000,0.328000,0.417000,0.417000
7,wpli,alpha,0.588000,0.169000,0.368000,0.357000,0.250000,0.351000,beta,0.738000,0.154000,0.667000,0.307000,0.600000,0.221000,alpha,0.733000,0.150000,0.654000,0.325000,0.567000,0.330000,alpha,0.668000,0.180000,0.552000,0.332000,0.533000,0.295000,theta,0.533000,0.210000,0.417000,0.282000,0.417000,0.190000
8,wpli2_debiased,theta,0.688000,0.116000,0.552000,0.189000,0.571000,0.409000,delta,0.688000,0.129000,0.632000,0.367000,0.400000,0.297000,theta,0.635000,0.154000,0.500000,0.324000,0.367000,0.253000,theta,0.583000,0.152000,0.444000,0.233000,0.533000,0.339000,beta,0.617000,0.108000,0.522000,0.294000,0.500000,0.209000


In [49]:
summary_results_df.to_excel("connectivity_performance_data_types.xlsx")  

#### Load table and generate the results for Statistical Features

In [10]:
%%capture
#test
save_folder = '/user/home/ko20929/work/RBD_using_custom_package/Execute New Experiments/Baseline_Extensions/Combine_Max_Freq_Power_Band/'
data_types = ['REM', 'N1', 'N2', 'N3', 'Wake']

# Create summary results dictionary that will be filled and populated
summary_results_dict = {}
summary_results_dict['method'] = ['stat features']
for data_type in data_types:
    # summary_results_dict[data_type + '_model'] = []
    summary_results_dict[data_type + '_acc'] = []
    summary_results_dict[data_type + '_acc' + '_std'] = []

for data_type in data_types:
    file_name = data_type + 'combined_max_freq_plus_BPW_TS_Fresh_Minimal_feats_results.pkl'
    results_df = joblib.load(save_folder + file_name)[1][data_type]['regions_combined']
    results_df = results_df[ ['random_state', 'y_true'] + [col for col in results_df.columns if 'Ada' in col] ]

    res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)
    # This will get you best performing Ada Model and Value for each method for each data type
    best_acc = res_df.mean(axis = 0)[1:].max()
    best_model = res_df.mean(axis = 0)[1:].idxmax()

    #Use the summary res_df to identify best model, keep raw results of best model to calculate the true std
    results_df_trimmed = results_df[ list(results_df.columns)[:2] + [col for col in results_df.columns if best_model[:-4] in col] ]
    per_fold_metrics = calculate_results.calc_per_fold_metrics(results_df)
    overalled_weighted_df = calculate_results.overall_weighted_metrics(per_fold_metrics)
    
    std_val = overalled_weighted_df[best_model + '_std'].values[0]

    # summary_results_dict[data_type + '_model'].append(best_model[:-10])
    summary_results_dict[data_type + '_acc'].append(best_acc)
    summary_results_dict[data_type + '_acc' + '_std'].append(std_val)


In [11]:
summary_results_df = pd.DataFrame(summary_results_dict)
summary_results_df = summary_results_df.round(3)
summary_results_df

,method,REM_acc,REM_acc_std,N1_acc,N1_acc_std,N2_acc,N2_acc_std,N3_acc,N3_acc_std,Wake_acc,Wake_acc_std
0,stat features,0.638,0.19,0.775,0.11,0.579,0.187,0.607,0.224,0.783,0.169


#### Load table and generate the results for Statistical Features and Connectivity Combined

In [25]:
best_model

'Left Parietal_Occipital_Ada_B_acc'

In [26]:
results_df_trimmed

,random_state,y_true,Left Parietal_Occipital_Ada_B_y_preds
0,1,"[[0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 1, ...","[[1, 1, 0, 0, 0, 0, 0, 1, 1], [1, 0, 0, 0, 0, ..."
1,2,"[[0, 0, 0, 0, 0, 0, 0, 1, 1], [0, 0, 0, 1, 1, ...","[[1, 1, 0, 0, 1, 1, 1, 1, 1], [0, 1, 0, 0, 0, ..."


In [30]:
%%capture

folder = '/user/home/ko20929/work/RBD_using_custom_package/Execute New Experiments/Connectivity/view_combined_results/'

f_names = ['bpw_plus_max_freqMinimal_connectivity_pli_gamma.pkl',
 'bpw_plus_max_freqMinimal_connectivity_pli_delta.pkl',
 'bpw_plus_max_freqMinimal_connectivity_pli_sigma.pkl',
 'bpw_plus_max_freqMinimal_connectivity_pli_alpha.pkl',
 'bpw_plus_max_freqMinimal_connectivity_pli_beta.pkl',
 'bpw_plus_max_freqMinimal_connectivity_pli_theta.pkl']

bands = [ 'gamma', 'delta', 'sigma', 'alpha' , 'beta' , 'theta']


# Create summary results dictionary that will be filled and populated
summary_results_dict = {}
summary_results_dict['band'] = []
for data_type in data_types:
    summary_results_dict[data_type + '_model'] = []
    summary_results_dict[data_type + '_acc'] = []
    summary_results_dict[data_type + '_acc' + '_std'] = []

error_list = []

for band in bands:
    error_dict[band] = []
    
    summary_results_dict['band'].append(band)
    f_name = 'bpw_plus_max_freqMinimal_connectivity_pli_' + band + '.pkl' 
    load_path = folder + f_name
    results_dict = joblib.load(folder + f_name)[1]


    
    for data_type in data_types:
        try:
            
            results_df = results_dict[data_type]['regions_combined']

            results_df = results_df[ ['random_state', 'y_true'] + [col for col in results_df.columns if 'Ada' in col] ]
            
            # raw_results[method][data_type] = results_df

        
            
            res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)
            # This will get you best performing Ada Model and Value for each band for each data type
            best_acc = res_df.mean(axis = 0)[1:].max()
            best_model = res_df.mean(axis = 0)[1:].idxmax()

            #Use the summary res_df to identify best model, keep raw results of best model to calculate the true std
            results_df_trimmed = results_df[ list(results_df.columns)[:2] + [col for col in results_df.columns if best_model[:-4] in col] ]
            per_fold_metrics = calculate_results.calc_per_fold_metrics(results_df_trimmed)
            overalled_weighted_df = calculate_results.overall_weighted_metrics(per_fold_metrics)
            
            std_val = overalled_weighted_df[best_model + '_std'].values[0]

            summary_results_dict[data_type + '_model'].append(best_model[:-10])
            summary_results_dict[data_type + '_acc'].append(best_acc)
            summary_results_dict[data_type + '_acc' + '_std'].append(std_val)
        except:
            error_list.append(band, data_type)

In [31]:
summary_results_df = pd.DataFrame(summary_results_dict)
summary_results_df = summary_results_df.round(3)
summary_results_df

,band,REM_model,REM_acc,REM_acc_std,N1_model,N1_acc,N1_acc_std,N2_model,N2_acc,N2_acc_std,N3_model,N3_acc,N3_acc_std,Wake_model,Wake_acc,Wake_acc_std
0,gamma,Right Frontal_Right Central,0.625,0.138,Frontal_Occipital,0.913,0.081,Left Parietal_Occipital,0.586,0.227,Left Central_Right Central,0.649,0.143,Frontal_Left Parietal,0.733,0.171
1,delta,Central_Left Temporal,0.575,0.151,Parietal_Occipital,0.712,0.149,Right Temporal_Occipital,0.558,0.190,Left Frontal_Left Temporal,0.576,0.269,Frontal_Left Frontal,0.683,0.091
2,sigma,Right Frontal_Central,0.538,0.160,Left Frontal_Occipital,0.725,0.147,Frontal_Left Central,0.557,0.174,Right Central_Occipital,0.621,0.130,Left Frontal_Left Parietal,0.683,0.050
3,alpha,Prefrontal_Left Central,0.625,0.195,Prefrontal_Parietal,0.675,0.151,Left Temporal_Occipital,0.733,0.129,Central_Left Central,0.693,0.205,Left Frontal_Occipital,0.667,0.075
4,beta,Right Frontal_Left Parietal,0.587,0.169,Right Frontal_Left Temporal,0.888,0.143,Frontal_Right Temporal,0.542,0.161,Frontal_Left Frontal,0.612,0.180,Frontal_Right Parietal,0.717,0.077
5,theta,Prefrontal_Left Parietal,0.662,0.149,Left Frontal_Occipital,0.625,0.187,Right Central_Right Temporal,0.611,0.177,Left Central_Left Temporal,0.643,0.136,Prefrontal_Frontal,0.800,0.196
